In [1]:
# -*-coding:utf-8 -*-
# import all the libraries 
# python==3.8; jupyterlab==3.0.12; lumicks.pylake==0.8.1; matplotlib==3.3.4; more-itertools==8.7.0;
# npTDMS==1.1.0; numpy==1.20.1; opencv-python==4.5.1.48; pandas==1.2.3; scipy==1.6.1; tifffile==2021.3.5
import os
import numpy as np
import cv2
import pylab as pl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import collections  as mc
from matplotlib import colors as mcolors
import pandas as pd
from nptdms import TdmsFile
from scipy.signal import savgol_filter
import tifffile as tif
from scipy import interpolate
from scipy.interpolate import interp1d
%matplotlib widget 

In [27]:
# this code intends to read and process all the files within this folder (ie. save all the .tdms file as .tiff)
# Folder Path
path = input('please type in the file path:' )

please type in the file path: E:\OneDrive - Vrije Universiteit Amsterdam\T7 gp2.5 MS_2021\Data&CodeRepository\Fig3_reanalysis_11Mar2022\mutantSSB\18pN


In [3]:
# # reads in 3 channel 16-bit TIFFs 
# img = tif.imread(image_path)

# background = 4
# vmax = 15
# plt.imshow(img.astype('uint8'),cmap='gray',vmax = vmax,aspect ="auto")
# plt.show()

In [11]:
# Define a function to read the tdms file and save as tiff image
def read_tdms_as_tiff(file_path):

    # read the meta data
    metadata = TdmsFile.read_metadata(file_path)
    # print(metadata.properties)
    width = metadata.properties['Pixels per line']
    px_size = float(metadata.properties['Scan Command.Scan Command.scanning_axes.0.pix_size_nm'])
    px_dwell_time = float(metadata.properties['Scan Command.PI Fast Scan Command.pixel_dwell_time_ms'])
    inter_frame_time = float(metadata.properties['Scan Command.PI Fast Scan Command.inter_frame_wait_time_ms'])

    tdms_file = TdmsFile(file_path)
    kymo_time = tdms_file['Data']['Time (ms)'][:]
    kymo_time = np.array([int(i) for i in kymo_time])
    kymo_position = tdms_file['Data']['Actual position X (um)'][:]
    kymo_position = np.array([int(i) for i in kymo_position])
    height = len(kymo_time)/width
    time_per_line = kymo_time[-1]/height # this is calculated by (time_per_line = acquisition time/all lines)

    chn_r = tdms_file['Data']['Pixel ch 1'][:]
    chn_r = np.array([int(i) for i in chn_r])
    chn_g = tdms_file['Data']['Pixel ch 2'][:]
    chn_g = np.array([int(i) for i in chn_g])
    chn_b = tdms_file['Data']['Pixel ch 3'][:]
    chn_b = np.array([int(i) for i in chn_b])

    chn_rgb = np.vstack((chn_r, chn_g, chn_b)).T
    img = chn_rgb.reshape((int(height), int(width), 3))
    img = img.transpose((1, 0, 2))
    img = img.astype(np.uint16)

    # split channels
    r,g,b = cv2.split(img)
    import json
    metadata_tag = json.dumps({"pixel size/nm": px_size, "pixel dwell time/ms": px_dwell_time,"inter-frame wait time/ms":inter_frame_time})
    extra_tags = [("MicroManagerMetadata", 's', 0, metadata_tag, True)]

    # save .tdms file to tif image, along with the metadata
    tif.imwrite(file_path[:-5] + '.tiff', img,extratags=extra_tags)
    
    # access the metadata
    # frames = tif.TiffFile(file_path[:-5] + '.tiff')
    # page = frames.pages[0]
    # print(page.tags["MicroManagerMetadata"].value)

In [5]:
# test the function
# # read_tdms_as_tiff(file_path)
# # access the metadata
# # frames = tif.TiffFile(file_path[:-5] + '.tiff')
# # page = frames.pages[0]
# # print(page.tags["MicroManagerMetadata"].value)

In [28]:
# iterate through all file
for file in os.listdir(path):
# Check whether file is in .tdms format or not
    if file.endswith(".tdms"):
        file_path = os.path.join(path, file) 
        
        # call read all file function
        read_tdms_as_tiff(file_path)

In [7]:
break:be careful with the below codes

SyntaxError: invalid syntax (3825573833.py, line 1)

In [49]:
# this code intends to read and process all the files within this folder (ie. save all the .tdms file as .tiff)
# Folder Path
path = input('please type in the file path:' )

please type in the file path: C:\Users\KTS260\Desktop\test


In [60]:
# save 156-bit tiff file as 8-bit tiff file
# Define a function to read the tdms file and save as tiff image
def save_16tiff_as_8tiff(file_path):
    img16 = tif.imread(file_path)
    img8 = cv2.convertScaleAbs(img16, alpha=0.03)
    
#     img8 = (img16/256).astype('uint8')
    tif.imwrite(file_path[:-5] + '.tiff', img8)
    fig = plt.figure()
    img8 = cv2.cvtColor(img8, cv2.COLOR_BGR2RGB)
    plt.imshow(img8.astype('uint16'),vmax = 15)
    plt.savefig(file_path[:-5] +'.eps', format='eps')

In [33]:
print(file_path)

E:\OneDrive - Vrije Universiteit Amsterdam\T7 gp2.5 MS_2021\Data&CodeRepository\Fig3_reanalysis_11Mar2022\mutantSSB\18pN\20210309-172556 test mutant gp2.5-Atto647N + L-DNA #014-001.tdms


In [61]:
# iterate through all file
for file in os.listdir(path):
# Check whether file is in .tdms format or not
    if file.endswith(".tif"):
        file_path = os.path.join(path, file) 
        print(file_path)
        # call read all file function
        save_16tiff_as_8tiff(file_path)

C:\Users\KTS260\Desktop\test\Fig1C_left.tif


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
break:be careful with the below codes

In [8]:
background = 4

In [9]:
# Define a function to read the tdms file and save as tiff image, with background substraction
def read_tdms_as_tiff_sub_bg(file_path):

    # read the meta data
    metadata = TdmsFile.read_metadata(file_path)
    # print(metadata.properties)
    width = metadata.properties['Pixels per line']
    px_size = float(metadata.properties['Scan Command.Scan Command.scanning_axes.0.pix_size_nm'])
    px_dwell_time = float(metadata.properties['Scan Command.PI Fast Scan Command.pixel_dwell_time_ms'])
    inter_frame_time = float(metadata.properties['Scan Command.PI Fast Scan Command.inter_frame_wait_time_ms'])

    tdms_file = TdmsFile(file_path)
    kymo_time = tdms_file['Data']['Time (ms)'][:]
    kymo_time = np.array([int(i) for i in kymo_time])
    kymo_position = tdms_file['Data']['Actual position X (um)'][:]
    kymo_position = np.array([int(i) for i in kymo_position])
    height = len(kymo_time)/width
    time_per_line = kymo_time[-1]/height # this is calculated by (time_per_line = acquisition time/all lines)

    chn_r = tdms_file['Data']['Pixel ch 1'][:]
    chn_r = np.array([int(i) for i in chn_r])
    chn_g = tdms_file['Data']['Pixel ch 2'][:]
    chn_g = np.array([int(i) for i in chn_g])
    chn_b = tdms_file['Data']['Pixel ch 3'][:]
    chn_b = np.array([int(i) for i in chn_b])

    chn_rgb = np.vstack((chn_r, chn_g, chn_b)).T
    img = chn_rgb.reshape((int(height), int(width), 3))
    img = img.transpose((1, 0, 2))
    img = img.astype(np.uint16)

    # split channels
    r,g,b = cv2.split(img)
    ret,thresh = cv2.threshold(g,background,255,cv2.THRESH_TOZERO)
    import json
    metadata_tag = json.dumps({"pixel size/nm": px_size, "pixel dwell time/ms": px_dwell_time,"inter-frame wait time/ms":inter_frame_time})
    extra_tags = [("MicroManagerMetadata", 's', 0, metadata_tag, True)]

    # save .tdms file to tif image, along with the metadata
    tif.imwrite(file_path[:-5] + '_background_substraction.tiff',thresh,extratags=extra_tags)
    
    # access the metadata
    # frames = tif.TiffFile(file_path[:-5] + '.tiff')
    # page = frames.pages[0]
    # print(page.tags["MicroManagerMetadata"].value)

In [10]:
# iterate through all file
for file in os.listdir(path):
# Check whether file is in .tdms format or not
    if file.endswith(".tdms"):
        file_path = os.path.join(path, file) 
        
        # call read all file function
        read_tdms_as_tiff_sub_bg(file_path)